In [3]:
from ember import Config, Paths, InitialCondition, StrainParameters, General, Times, TerminationCondition, ConcreteConfig, Grid, PositionControl,General, Debug
import matplotlib.pyplot as plt

output = 'run/ex_diffusion9'
import numpy as np
import matplotlib.pyplot as plt
import h5py
import cantera as ct
import os

from ember import _ember

mamba activate ember-build1   


In [4]:
def plot_analysis(struct_file, species_to_plot, species_index):
    struct = h5py.File(struct_file, 'r')
    x = struct['x'][:]
    Y = struct['Y'][:]
    V = struct['V'][:]
    T = struct['T'][:]
    
    print(f"Max temperature: {np.max(T)}")
    rho = struct['rho'][:]
    
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(8, 20), dpi=200)
    
    ax1.plot(x, T, label='T')
    ax1.set_xlabel('Position [m]')
    ax1.set_ylabel('Temperature [K]')
    ax1.set_title('Temperature Profile')
    ax1.legend()
    ax1.grid()
    
    ax2.plot(x, V, label='V')
    ax2.set_xlabel('Position [m]')
    ax2.set_ylabel('Velocity [m/s]')
    ax2.set_title('Velocity Profile')
    ax2.legend()
    ax2.grid()
    
    ax3.plot(x, rho, label='rho')
    ax3.set_xlabel('Position [m]')
    ax3.set_ylabel('Density [kg/m3]')
    ax3.set_title('Density Profile')
    ax3.legend()
    
    ax3.grid()
    
    for i, index in enumerate(species_index):
        ax4.plot(x, Y[index, :], label=species_to_plot[i])  
    ax4.set_xlabel('Position [m]')
    ax4.set_ylabel('Mass Fraction')
    ax4.set_title('Mass Fraction Profile')
    ax4.legend()
    
    ax4.grid()
    
    plt.show()

"""
Visualization tools for PyEmber flame solutions
"""
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from typing import Optional, List, Tuple

class FlameVisualizer:
    """
    Visualization tools for flame solutions
    """
    def __init__(self, flame):
        self.flame = flame
        self.fig = None
        self._animation = None
        self.history = {
            't': [],
            'T': [],
            'Y': [],
            'V': [],
            'U': []
        }

    def save_state(self):
        """Save current state for animation"""
        # self.history['t'].append(self.flame.t)
        self.history['T'].append(self.flame.T.copy())
        self.history['Y'].append(self.flame.Y.copy())
        self.history['V'].append(self.flame.V.copy())
        self.history['U'].append(self.flame.U.copy())

    def plot_current_state(self, species_names: Optional[List[str]] = None, gas=None, title="Flame structure"):
        """
        Plot current flame state
        
        Args:
            species_names: List of species to plot (if None, plots major species)
        """
        # check if flame as attribute grid
        if not hasattr(self.flame, 'grid'):
            x = self.flame.x * 1000  # Convert to mm
        else:
            x = self.flame.grid.x * 1000  # Convert to mm
        
        # Create figure with 3 subplots
        if self.fig is None:
            self.fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12))
            self.fig.suptitle(title)
        else:
            ax1, ax2, ax3 = self.fig.axes
            for ax in (ax1, ax2, ax3):
                ax.clear()

        # Temperature profile
        ax1.plot(x, self.flame.T, 'r-', label='Temperature')
        ax1.set_ylabel('Temperature [K]')
        ax1.legend()
        ax1.grid(True)

        # Species profiles
        if species_names is None:
            # Plot major species (Y > 0.01 anywhere)
            mask = np.max(self.flame.Y, axis=1) > 0.01
            species_indices = np.where(mask)[0]
            if gas is None:
                species_names = [self.flame.gas.species_name(k) for k in species_indices]
            else:
                species_names = [gas.species_name(k) for k in species_indices]
        else:
            if gas is None:
                species_indices = [self.flame.gas.species_index(name) 
                                for name in species_names]
            else:
                species_indices = [gas.species_index(name) 
                             for name in species_names]

        for k, name in zip(species_indices, species_names):
            ax2.plot(x, self.flame.Y[k], label=name)
        ax2.set_ylabel('Mass Fraction')
        ax2.legend()
        ax2.grid(True)

        # Velocity profile
        ax3.plot(x, self.flame.U, 'b-', label='Mass Flux')
        ax3.set_xlabel('Position [mm]')
        ax3.set_ylabel('Mass Flux [kg/m²/s]')
        ax3.legend()
        ax3.grid(True)

        plt.tight_layout()
        plt.show()

    def create_animation(self, species_names: Optional[List[str]] = None,
                        interval: int = 50) -> FuncAnimation:
        """
        Create animation of flame evolution
        
        Args:
            species_names: List of species to animate
            interval: Time between frames in milliseconds
            
        Returns:
            matplotlib.animation.FuncAnimation object
        """
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12))
        fig.suptitle('Flame Evolution')

        x = self.flame.grid.x * 1000  # mm
        
        # Set up species to plot
        if species_names is None:
            mask = np.max(self.flame.Y, axis=1) > 0.01
            species_indices = np.where(mask)[0]
            species_names = [self.flame.gas.species_name(k) for k in species_indices]
        else:
            species_indices = [self.flame.gas.species_index(name) 
                             for name in species_names]
            print(species_indices)

        # Temperature limits
        T_min = min(np.min(T) for T in self.history['T'])
        T_max = max(np.max(T) for T in self.history['T'])
        
        # Species limits
        Y_max = {k: max(np.max(Y[k]) for Y in self.history['Y'])
                 for k in species_indices}
        
        # Velocity limits
        V_min = min(np.min(V) for V in self.history['V'])
        V_max = max(np.max(V) for V in self.history['V'])

        def animate(frame):
            # Clear axes
            for ax in (ax1, ax2, ax3):
                ax.clear()
                ax.grid(True)

            # Temperature
            ax1.plot(x, self.history['T'][frame], 'r-')
            ax1.set_ylabel('Temperature [K]')
            ax1.set_ylim(T_min, T_max * 1.1)

            # Species
            for k, name in zip(species_indices, species_names):
                ax2.plot(x, self.history['Y'][frame][k], label=name)
            ax2.set_ylabel('Mass Fraction')
            ax2.legend()
            #ax2.set_ylim(0, max(Y_max.values()) * 1.1)

            # Velocity
            ax3.plot(x, self.history['V'][frame], 'b-')
            ax3.set_xlabel('Position [mm]')
            ax3.set_ylabel('Mass Flux [kg/m²/s]')
            ax3.set_ylim(V_min * 1.1, V_max * 1.1)

            # Time stamp
            fig.suptitle(f'Flame Evolution (t = {self.history["t"][frame]:.3f} s)')

        self._animation = FuncAnimation(
            fig, animate, frames=len(self.history['t']),
            interval=interval, blit=False
        )
        
        return self._animation

    def save_animation(self, filename: str, fps: int = 20):
        """
        Save animation to file
        
        Args:
            filename: Output filename (.mp4 or .gif)
            fps: Frames per second
        """
        if self._animation is None:
            self.create_animation()
            
        self._animation.save(filename, fps=fps)

In [5]:
# get index for CH4, O2, N2, H2O, CO2, CO
gas = ct.Solution('gri30.yaml')
names = gas.species_names
ch4_index = names.index('CH4')
o2_index = names.index('O2')
n2_index = names.index('N2')
h2o_index = names.index('H2O')
co2_index = names.index('CO2')
co_index = names.index('CO')
oh_index = names.index('OH')


In [ ]:

from ember import Config, Paths, InitialCondition, StrainParameters, General, Times, TerminationCondition, ConcreteConfig, Grid, PositionControl,General, Debug
import matplotlib.pyplot as plt

output = 'run/ex_diffusion9'
import numpy as np
import matplotlib.pyplot as plt
import h5py
import cantera as ct
import os

from ember import _ember
conf = Config(
    Paths(outputDir='run/ex_diffusion11'),
    General(nThreads=1,
            chemistryIntegrator='qss'),
    InitialCondition(Tfuel=600,
                     Toxidizer=1200,
                     centerWidth=0.0,
                     equilibrateCounterflow=False,
                     flameType='diffusion',
                     slopeWidth=0.0,
                     xLeft=-0.02,
                     pressure=101325,
                     xRight=0.02,
                     nPoints=100),
    StrainParameters(final=100,
                     initial=100),
    Times(globalTimestep=1e-05,
          profileStepInterval=20),
    TerminationCondition(abstol=0.0,
                         dTdtTol=0,
                         steadyPeriod=1.0,
                         tEnd=0.08,
                         tolerance=0.0),
    Debug(veryVerbose=False),)


conf = ConcreteConfig(conf)

confString = conf.original.stringify()

if not os.path.isdir(conf.paths.outputDir):
    os.makedirs(conf.paths.outputDir, 0o0755)
confOutPath = os.path.join(conf.paths.outputDir, 'config')
if (os.path.exists(confOutPath)):
    os.unlink(confOutPath)
confOut = open(confOutPath, 'w')
confOut.write(confString)



solver = _ember.FlameSolver(conf)
solver.initialize()
# visualizer = FlameVisualizer(solver)




In [ ]:
prev_T = solver.T 
prev_Y = solver.Y
prev_x = solver.x

visualizer.save_state()

done = 0
i = 1
plot_freq = 10
print(solver.T[21:51])
while not done:
    done = solver.step()
    if i % 5 == 0:
        visualizer.save_state()
        
    # if i % plot_freq == 0:  
    #     plot_flame(solver, i, i*1e-4)
       
    i += 1
solver.finalize()

In [ ]:
confString = conf.original.stringify()

if not os.path.isdir(conf.paths.outputDir):
    os.makedirs(conf.paths.outputDir, 0o0755)
confOutPath = os.path.join(conf.paths.outputDir, 'config')
if (os.path.exists(confOutPath)):
    os.unlink(confOutPath)
confOut = open(confOutPath, 'w')
confOut.write(confString)

In [6]:

xhis = np.load('/Users/elotech/Documents/CODES/SCI-ML/oneDFlameSolver/ember/python/ember/examples/run/ex_diffusion9/xhistory copy.npy')
This = np.load('/Users/elotech/Documents/CODES/SCI-ML/oneDFlameSolver/ember/python/ember/examples/run/ex_diffusion9/thistory copy.npy')
Yhis = np.load('/Users/elotech/Documents/CODES/SCI-ML/oneDFlameSolver/ember/python/ember/examples/run/ex_diffusion9/yhistory copy.npy')

In [6]:


solver = _ember.FlameSolver(conf)
solver.initialize()
visualizer = FlameVisualizer(solver)

prev_T = solver.T 
prev_Y = solver.Y
prev_x = solver.x

visualizer.save_state()


In [7]:
def plot_flame(solver, iter=0, t=0):
    """
    Plot flame structure (temperature, species, velocity)
    """
    species_names=['CH4', 'O2', 'N2', 'H2O', 'CO2', 'CO', 'OH']
    title = 'Flame Structure'
    if not hasattr(solver, 'grid'):
        x = solver.x * 1000  # Convert to mm
    else:
        x = solver.grid.x * 1000  # Convert to mm

    # Create figure with 3 subplots

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12))
    fig.suptitle(f'{title} at t = {t:.3f} s')


    # Temperature profile
    ax1.plot(x, solver.T, 'r-', label='Temperature')
    ax1.set_ylabel('Temperature [K]')
    ax1.legend()
    ax1.grid(True)

    # Species profiles
    if species_names is None:
        # Plot major species (Y > 0.01 anywhere)
        mask = np.max(solver.Y, axis=1) > 0.01
        species_indices = np.where(mask)[0]
        if gas is None:
            species_names = [solver.gas.species_name(k) for k in species_indices]
        else:
            species_names = [gas.species_name(k) for k in species_indices]
    else:
        if gas is None:
            species_indices = [solver.gas.species_index(name) 
                            for name in species_names]
        else:
            species_indices = [gas.species_index(name) 
                            for name in species_names]

    for k, name in zip(species_indices, species_names):
        ax2.plot(x, solver.Y[k], label=name)
    ax2.set_ylabel('Mass Fraction')
    ax2.legend()
    ax2.grid(True)

    # Velocity profile
    ax3.plot(x, solver.U, 'b-', label='Mass Flux')
    ax3.set_xlabel('Position [mm]')
    ax3.set_ylabel('Mass Flux [kg/m²/s]')
    ax3.legend()
    ax3.grid(True)

    plt.tight_layout()
    plt.savefig(f'flame_{iter}.png')

In [ ]:
done = solver.step()

In [ ]:
done = 0
i = 1
plot_freq = 10
print(solver.T[21:51])
while not done:
    done = solver.step()
    if i % 5 == 0:
        visualizer.save_state()
        
    # if i % plot_freq == 0:  
    #     plot_flame(solver, i, i*1e-4)
       
    i += 1
solver.finalize()

In [ ]:
plt.plot(solver.x, visualizer.history['V'][1], label="initial")
plt.title("Velocity Profile (CPP)")
# plt.plot(solver.x, visualizer.history['V'][800], label="final")

In [ ]:
plt.plot(solver.x, solver.U)

In [ ]:
plt.plot(solver.x, solver.T)

In [129]:
# save history 
cpp_history = visualizer.history

In [130]:
# save dictionary to file
import pickle
with open('cpp_history.pkl', 'wb') as f:
    pickle.dump(cpp_history, f)

In [ ]:
plt.plot(solver.x, solver.U)

In [ ]:
plt.plot(solver.x, solver.V)

In [ ]:
plt.plot(solver.x, visualizer.history['T'][-1])

In [ ]:
plt.plot(solver.x, solver.rhoD[ch4_index, :])
plt.plot(solver.x, solver.rhoD[o2_index, :])
plt.plot(solver.x, solver.rhoD[n2_index, :])

In [134]:
struct_file = 'run/ex_diffusion10/profNow.h5'
struct = h5py.File(struct_file, 'r')

x = struct['x'][:]
Y = struct['Y'][:]
V = struct['V'][:]
T = struct['T'][:]

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(8, 6), dpi=200)
plt.plot(prev_x, prev_T, label='cpp(0.0)', color='red')
plt.plot(solver.x, solver.T, label='cpp (0.08)', linestyle='dashed')
plt.plot(x, T, label='cpp (0.01)', linestyle='dashdot')
plt.plot(xhis[0], This[0], label='python (0.0)', linestyle='dotted')
plt.plot(xhis[-1], This[-1], label='python (0.01)', linestyle='dotted')
plt.legend()
plt.title('Temperature Profile, CPP')
plt.show() 


In [ ]:
plt.plot(solver.x, solver.Y[ch4_index,:], label='final CH4')
plt.plot(prev_x, prev_Y[ch4_index,:], label='initial CH4',linestyle='--')
plt.plot(prev_x, prev_Y[o2_index,:], label='initial O2', linestyle='--')
plt.plot(solver.x, solver.Y[o2_index,:], label='final O2')
plt.plot(solver.x, solver.Y[n2_index,:], label='final N2', linestyle='--')
plt.plot(prev_x, prev_Y[n2_index,:], label='initial N2')

plt.plot(solver.x, solver.Y[h2o_index,:], label='H2O')
plt.plot(solver.x, solver.Y[co2_index,:], label='CO2')

plt.plot(solver.x, solver.Y[co_index,:], label='CO')

plt.plot(solver.x, solver.Y[oh_index,:], label='OH')

plt.legend()
plt.title("Correct Evolution of Species (CPP)")
plt.savefig('species_evolution.png')
plt.show()

In [15]:
# get index for CH4, O2, N2, H2O, CO2, CO
gas = ct.Solution('gri30.yaml')
names = gas.species_names
ch4_index = names.index('CH4')
o2_index = names.index('O2')
n2_index = names.index('N2')
h2o_index = names.index('H2O')
co2_index = names.index('CO2')
co_index = names.index('CO')
oh_index = names.index('OH')


In [ ]:

plt.plot(prev_x, prev_T, label='initial T', color='red')
plt.plot(x, T, label='final T', linestyle='dashed')
plt.legend()
plt.title('Temperature Profile, CPP')
plt.show() 


In [ ]:

plot_analysis(struct_file, ['CH4', 'O2', 'H2O', 'CO2', 'OH'], [ch4_index, o2_index, h2o_index, co2_index, oh_index])

In [ ]:
struct_file_2 = 'run/ex_diffusion7/prof000201.h5'
plot_analysis(struct_file_2, ['CH4', 'O2', 'H2O', 'CO2', 'CO'], [ch4_index, o2_index, h2o_index, co2_index, co_index])

In [ ]:
struct_file_3 = '../../../run/ex_diffusion6/prof000210.h5'
plot_analysis(struct_file_3, ['CH4', 'O2', 'H2O', 'CO2', 'CO'], [ch4_index, o2_index, h2o_index, co2_index, co_index])

In [1]:
from ember import ZeroDCombustion

from ember import Config, Paths, InitialCondition, StrainParameters, General, Times, TerminationCondition, ConcreteConfig, Grid, PositionControl,General, Debug
import matplotlib.pyplot as plt
import numpy as np
import cantera as ct

gas = ct.Solution('gri30.yaml')
names = gas.species_names
ch4_index = names.index('CH4')
o2_index = names.index('O2')
n2_index = names.index('N2')
h2o_index = names.index('H2O')
co2_index = names.index('CO2')
co_index = names.index('CO')
oh_index = names.index('OH')


In [2]:

# Create and configure the solver
conf = ConcreteConfig(Config(
    Paths(outputDir='run/ex_diffusion12'),
    General(nThreads=1,
            chemistryIntegrator='boostRK'),
)
    )


jm = 49


In [3]:
solver = ZeroDCombustion(conf)  # Use your existing ConfigOptions


In [4]:
solver.initialize()

Creating integrator of type: boostRK
Creating Boost RK integrator
Setting BoostRK options
Initializing BoostRK
Integrator created successfully
ZeroDCombustion initialized successfully


In [5]:
# Set initial state
T0 = 1000  # Initial temperature in K
Y0 = np.zeros(len(gas.species_names))  # Initial mass fractions
Y0[ch4_index] = 0.1  # Set fuel mass fraction
Y0[o2_index] = 0.9  # Set oxidizer mass fraction



In [6]:
solver.set_state(T0, Y0)

In [7]:
# Integration with different integrators
solver.set_integrator_type('cvode')  # Start with CVODE



: 

In [ ]:
solver.integrate_to_time(2e-6)  # Next microsecond

: 

In [ ]:





solver.set_integrator_type('qss')  # Switch to QSS
solver.integrate_to_time(2e-6)  # Next microsecond

# Get results
T = solver.T  # Current temperature
Y = solver.Y  # Current mass fractions
wdot = solver.reaction_rates  # Current reaction rates
qdot = solver.heat_release_rate  # Current heat release rate